<a href="https://colab.research.google.com/github/Ashwini1505-png/codealpha/blob/main/Socialmediaplatform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install django
!django-admin startproject social_project .
!python manage.py startapp core
print("Django installed and project created!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 56.0 MB/s eta 0:00:00
Django installed and project created!


In [2]:
# This script automatically configures your settings for Colab
with open('social_project/settings.py', 'r') as f:
    content = f.read()

# Allow external access
content = content.replace("ALLOWED_HOSTS = []", "ALLOWED_HOSTS = ['*']")

# Register your 'core' app
content = content.replace(
    "'django.contrib.staticfiles',",
    "'django.contrib.staticfiles',\n    'core',"
)

with open('social_project/settings.py', 'w') as f:
    f.write(content)

print("Settings updated successfully for Google Colab!")




Settings updated successfully for Google Colab!


In [3]:
%%writefile core/models.py
from django.db import models
from django.contrib.auth.models import User

# 1. User Profile
class Profile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    bio = models.TextField(max_length=500, blank=True)
    followers = models.ManyToManyField(User, related_name='following', blank=True)

    def __str__(self):
        return self.user.username

# 2. Posts
class Post(models.Model):
    author = models.ForeignKey(User, on_delete=models.CASCADE)
    content = models.TextField(max_length=280)
    created_at = models.DateTimeField(auto_now_add=True)
    likes = models.ManyToManyField(User, related_name='liked_posts', blank=True)

    def __str__(self):
        return f"{self.author.username}'s post at {self.created_at}"

# 3. Comments
class Comment(models.Model):
    post = models.ForeignKey(Post, on_delete=models.CASCADE, related_name='comments')
    author = models.ForeignKey(User, on_delete=models.CASCADE)
    text = models.TextField(max_length=200)
    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"Comment by {self.author.username}"

Overwriting core/models.py


In [4]:
%%writefile core/admin.py
from django.contrib import admin
from .models import Profile, Post, Comment

admin.site.register(Profile)
admin.site.register(Post)
admin.site.register(Comment)


Overwriting core/admin.py


In [6]:
# 1. Build the database
!python manage.py makemigrations core
!python manage.py migrate

# 2. Create the Admin user using Python
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "social_project.settings")
# Allow synchronous database access in potentially async environments like Colab
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.contrib.auth.models import User
if not User.objects.filter(username='admin').exists():
    User.objects.create_superuser('admin', 'admin@example.com', 'admin123')
    print("\n✅ Database ready! Superuser created.")
    print("Username: admin")
    print("Password: admin123")
else:
    print("\n✅ Database ready! Admin user already exists.")


No changes detected in app 'core'
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, core, sessions
Running migrations:
  No migrations to apply.

✅ Database ready! Superuser created.
Username: admin
Password: admin123


In [7]:
from google.colab import output
import threading

# Run the Django server in the background
def run_server():
    !python manage.py runserver 0.0.0.0:8000

server_thread = threading.Thread(target=run_server)
server_thread.start()

print("🚀 Your server is starting...")
print("Click the link below to open your Mini Social Media Backend:")

# This creates a secure link to your Colab server
output.serve_kernel_port_as_window(8000)




🚀 Your server is starting...
Click the link below to open your Mini Social Media Backend:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [8]:
%%writefile core/views.py
from django.shortcuts import render, redirect
from .models import Post

def feed(request):
    # Handle creating a new post
    if request.method == 'POST':
        content = request.POST.get('content')
        if content and request.user.is_authenticated:
            Post.objects.create(author=request.user, content=content)
        return redirect('feed')

    # Get all posts to show on the page
    posts = Post.objects.all().order_by('-created_at')
    return render(request, 'core/feed.html', {'posts': posts})


Overwriting core/views.py


In [9]:
%%writefile core/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.feed, name='feed'),
]


Writing core/urls.py


In [10]:
%%writefile social_project/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('core.urls')),  # Links the core app to our homepage
]

Overwriting social_project/urls.py


In [16]:
import os

# Create the folders for our HTML files safely
os.makedirs('core/templates/core', exist_ok=True)

html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Mini Social Media</title>
    <!-- Tailwind CSS for styling -->
    <script src="[https://cdn.tailwindcss.com](https://cdn.tailwindcss.com)"></script>
    <link rel="stylesheet" href="[https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css](https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css)">
</head>
<body class="bg-slate-100 p-8 font-sans">
    <div class="max-w-2xl mx-auto">
        <div class="flex justify-between items-center mb-6">
            <h1 class="text-3xl font-bold text-slate-800">My Social Feed</h1>
            {% if user.is_authenticated %}
                <span class="text-slate-600 font-bold">Logged in as @{{ user.username }}</span>
            {% else %}
                <a href="/admin" class="bg-slate-800 text-white px-4 py-2 rounded-lg text-sm font-bold">Login to Post</a>
            {% endif %}
        </div>

        <!-- Post Creation Form -->
        {% if user.is_authenticated %}
        <form method="POST" class="bg-white p-5 rounded-xl shadow-sm mb-8 border border-slate-200">
            {% csrf_token %}
            <textarea name="content" class="w-full bg-slate-50 border border-slate-200 p-3 rounded-lg resize-none outline-none focus:ring-2 focus:ring-blue-500" placeholder="What's happening?" rows="3"></textarea>
            <div class="flex justify-end mt-3">
                <button type="submit" class="bg-blue-600 hover:bg-blue-700 text-white px-6 py-2 rounded-full font-bold transition-colors">Post</button>
            </div>
        </form>
        {% endif %}

        <!-- Posts Display -->
        <div class="space-y-4">
            {% for post in posts %}
            <div class="bg-white p-5 rounded-xl shadow-sm border border-slate-200">
                <div class="flex items-center gap-3 mb-3">
                    <div class="w-10 h-10 bg-indigo-500 rounded-full flex justify-center items-center text-white font-bold">
                        {{ post.author.username|make_list|first|upper }}
                    </div>
                    <div>
                        <div class="font-bold text-slate-800">{{ post.author.username }}</div>
                        <div class="text-xs text-slate-500">{{ post.created_at|timesince }} ago</div>
                    </div>
                </div>
                <p class="text-slate-800 text-lg">{{ post.content }}</p>
                <div class="flex gap-6 mt-4 pt-3 border-t border-slate-100 text-slate-500">
                    <button class="hover:text-rose-500 transition-colors"><i class="fa-regular fa-heart"></i> {{ post.likes.count }}</button>
                    <button class="hover:text-blue-500 transition-colors"><i class="fa-regular fa-comment"></i> {{ post.comments.count }}</button>
                </div>
            </div>
            {% empty %}
            <div class="text-center p-8 bg-white rounded-xl shadow-sm text-slate-500">
                No posts yet. Be the first to post!
            </div>
            {% endfor %}
        </div>
    </div>
</body>
</html>"""

# Write the HTML content to the file
with open('core/templates/core/feed.html', 'w') as f:
    f.write(html_content)

print("Template created successfully!")


Template created successfully!


In [17]:
import os

settings_path = 'social_project/settings.py'
with open(settings_path, 'r') as f:
    content = f.read()

# 1. Update Django settings to explicitly point to the templates folder
if "'DIRS': []" in content:
    content = content.replace("'DIRS': []", "'DIRS': [BASE_DIR / 'core' / 'templates']")
    with open(settings_path, 'w') as f:
        f.write(content)

# 2. 'Touch' the settings file to force the Django server to instantly restart
os.utime(settings_path, None)

# 3. Verify the file actually exists
if os.path.exists('core/templates/core/feed.html'):
    print("✅ Settings fixed and server restarted! Refresh your browser page now.")
else:
    print("❌ HTML file is missing. Please re-run Step 9 first, then run this again.")

✅ Settings fixed and server restarted! Refresh your browser page now.


In [18]:
import os

# 1. Fix the CSRF Error and Template Directories in settings.py
settings_path = 'social_project/settings.py'
with open(settings_path, 'r') as f:
    content = f.read()

if "'DIRS': []" in content:
    content = content.replace("'DIRS': []", "'DIRS': [BASE_DIR / 'core' / 'templates']")

if "CSRF_TRUSTED_ORIGINS" not in content:
    # This tells Django to trust Colab's URLs when you click "Post"
    content += "\nCSRF_TRUSTED_ORIGINS = ['https://*.colab.dev', 'https://*.googleusercontent.com']\n"

with open(settings_path, 'w') as f:
    f.write(content)

# 2. Fix the Rocket Ship by forcing the URLs to update
urls_content = """from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('core.urls')),
]"""
with open('social_project/urls.py', 'w') as f:
    f.write(urls_content)

# Force the Django server to instantly restart to apply all these fixes
os.utime(settings_path, None)

print("✅ All errors fixed! The server has been restarted.")
print("👉 Go back to your browser tab and refresh the page!")

✅ All errors fixed! The server has been restarted.
👉 Go back to your browser tab and refresh the page!


In [19]:
import os

# 1. Fix the CSRF Error by trusting Colab's URLs
settings_path = 'social_project/settings.py'
with open(settings_path, 'r') as f:
    content = f.read()

if "CSRF_TRUSTED_ORIGINS" not in content:
    content += "\nCSRF_TRUSTED_ORIGINS = ['https://*.colab.dev', 'https://*.googleusercontent.com']\n"

with open(settings_path, 'w') as f:
    f.write(content)

# 2. Fix the Rocket Ship by linking the URLs properly
urls_content = """from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('core.urls')),
]"""
with open('social_project/urls.py', 'w') as f:
    f.write(urls_content)

# 3. Force the Django server to instantly restart and apply fixes
os.utime(settings_path, None)

print("✅ All errors fixed! The server has been restarted.")
print("👉 Go back to your browser tab and refresh the page!")

✅ All errors fixed! The server has been restarted.
👉 Go back to your browser tab and refresh the page!


In [21]:
import os

# 1. Update views.py to add the signup logic
views_path = 'core/views.py'
with open(views_path, 'r') as f:
    views_content = f.read()

if 'def signup' not in views_content:
    new_imports = """from django.contrib.auth.forms import UserCreationForm
from django.contrib.auth import login
"""
    signup_view = """
def signup(request):
    if request.method == 'POST':
        form = UserCreationForm(request.POST)
        if form.is_valid():
            user = form.save()
            login(request, user) # Auto-login after signup
            return redirect('feed')
    else:
        form = UserCreationForm()
    return render(request, 'core/signup.html', {'form': form})
"""
    with open(views_path, 'w') as f:
        f.write(new_imports + views_content + signup_view)

# 2. Update urls.py to add the signup, login, and logout routes
urls_path = 'core/urls.py'
urls_content = """from django.urls import path
from django.contrib.auth import views as auth_views
from . import views

urlpatterns = [
    path('', views.feed, name='feed'),
    path('signup/', views.signup, name='signup'),
    path('login/', auth_views.LoginView.as_view(template_name='core/login.html'), name='login'),
    path('logout/', auth_views.LogoutView.as_view(next_page='feed'), name='logout'),
]
"""
with open(urls_path, 'w') as f:
    f.write(urls_content)

print("Views and URLs updated for signup, login, and logout!")


Views and URLs updated for signup, login, and logout!


# Task
Create the `core/templates/core/signup.html` and `core/templates/core/login.html` files, then provide instructions on how to test the signup and login functionality in the running Django application.

## Create Signup Template

### Subtask:
Create the 'core/templates/core/signup.html' file which will contain the HTML form for user registration.


**Reasoning**:
The subtask requires creating an HTML file for the signup page. I will use the `%%writefile` magic command to create the `core/templates/core/signup.html` file with the specified content.



In [22]:
import os

# Create the folders for our HTML files safely
os.makedirs('core/templates/core', exist_ok=True)

signup_html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Sign Up</title>
    <!-- Tailwind CSS for styling -->
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-slate-100 p-8 font-sans">
    <div class="max-w-md mx-auto bg-white p-8 rounded-xl shadow-sm border border-slate-200 mt-10">
        <h1 class="text-3xl font-bold text-slate-800 mb-6 text-center">Sign Up</h1>
        <form method="post" class="space-y-4">
            {% csrf_token %}
            {% for field in form %}
                <div>
                    <label for="{{ field.id_for_label }}" class="block text-sm font-medium text-slate-700">{{ field.label }}</label>
                    {{ field }}
                    {% if field.help_text %}
                        <p class="mt-1 text-sm text-slate-500">{{ field.help_text }}</p>
                    {% endif %}
                    {% for error in field.errors %}
                        <p class="mt-1 text-sm text-red-600">{{ error }}</p>
                    {% endfor %}
                </div>
            {% endfor %}
            <button type="submit" class="w-full bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold transition-colors">Sign Up</button>
        </form>
        <p class="mt-6 text-center text-sm text-slate-600">
            Already have an account? <a href="{% url 'login' %}" class="font-medium text-blue-600 hover:text-blue-500">Login</a>
        </p>
    </div>
</body>
</html>"""

# Write the HTML content to the file
with open('core/templates/core/signup.html', 'w') as f:
    f.write(signup_html_content)

print("Signup template created successfully!")

Signup template created successfully!


## Create Login Template

### Subtask:
Create the 'core/templates/core/login.html' file which will contain the HTML form for user login.

**Reasoning**:
The subtask requires creating an HTML file for the login page. I will use the `%%writefile` magic command (within a Python code block) to create the `core/templates/core/login.html` file with the specified content. This template will include a form for user login, styled with Tailwind CSS, and handle CSRF protection.

**Reasoning**:
Now I will create the `core/templates/core/login.html` file using a Python code block, embedding the HTML content directly into the file. This template will provide the user login interface, styled with Tailwind CSS, and handle Django's form rendering and CSRF protection.



In [23]:
import os

# Ensure the templates directory exists
os.makedirs('core/templates/core', exist_ok=True)

login_html_content = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Login</title>
    <!-- Tailwind CSS for styling -->
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-slate-100 p-8 font-sans">
    <div class="max-w-md mx-auto bg-white p-8 rounded-xl shadow-sm border border-slate-200 mt-10">
        <h1 class="text-3xl font-bold text-slate-800 mb-6 text-center">Login</h1>
        <form method="post" class="space-y-4">
            {% csrf_token %}
            {% for field in form %}
                <div>
                    <label for="{{ field.id_for_label }}" class="block text-sm font-medium text-slate-700">{{ field.label }}</label>
                    {{ field }}
                    {% if field.help_text %}
                        <p class="mt-1 text-sm text-slate-500">{{ field.help_text }}</p>
                    {% endif %}
                    {% for error in field.errors %}
                        <p class="mt-1 text-sm text-red-600">{{ error }}</p>
                    {% endfor %}
                </div>
            {% endfor %}
            <button type="submit" class="w-full bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold transition-colors">Login</button>
        </form>
        <p class="mt-6 text-center text-sm text-slate-600">
            Don't have an account? <a href="{% url 'signup' %}" class="font-medium text-blue-600 hover:text-blue-500">Sign Up</a>
        </p>
    </div>
</body>
</html>"""

# Write the HTML content to the file
with open('core/templates/core/login.html', 'w') as f:
    f.write(login_html_content)

print("Login template created successfully!")

Login template created successfully!


In [25]:
import os

# 1. Update urls.py to explicitly tell the Login page where to go next
urls_path = 'core/urls.py'
urls_content = """from django.urls import path
from django.contrib.auth import views as auth_views
from . import views

urlpatterns = [
    path('', views.feed, name='feed'),
    path('signup/', views.signup, name='signup'),
    # Added next_page='feed' to fix the login redirect error!
    path('login/', auth_views.LoginView.as_view(template_name='core/login.html', next_page='feed'), name='login'),
    path('logout/', auth_views.LogoutView.as_view(next_page='feed'), name='logout'),
]"""

with open(urls_path, 'w') as f:
    f.write(urls_content)

# 2. Add fallback redirect settings to settings.py just in case
settings_path = 'social_project/settings.py'
with open(settings_path, 'r') as f:
    content = f.read()

if "LOGIN_REDIRECT_URL" not in content:
    content += "\nLOGIN_REDIRECT_URL = '/'\n"
    content += "LOGOUT_REDIRECT_URL = '/'\n"

    with open(settings_path, 'w') as f:
        f.write(content)

# 3. Force server restart
os.utime(settings_path, None)
print("✅ Login redirect fixed! Server restarted.")
print("👉 Go back to your browser tab, refresh the page, and try logging in again!")

✅ Login redirect fixed! Server restarted.
👉 Go back to your browser tab, refresh the page, and try logging in again!


In [26]:
import os

feed_path = 'core/templates/core/feed.html'

# Open the HTML file
with open(feed_path, 'r') as f:
    content = f.read()

# Fix the broken Tailwind CSS and FontAwesome links
content = content.replace('"[https://cdn.tailwindcss.com](https://cdn.tailwindcss.com)"', '"https://cdn.tailwindcss.com"')
content = content.replace('"[https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css](https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css)"', '"https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"')

# Save the fixed file
with open(feed_path, 'w') as f:
    f.write(content)

print("✅ CSS Links fixed! Your site should look beautiful now.")
print("👉 Go back to your browser tab and refresh the page!")

✅ CSS Links fixed! Your site should look beautiful now.
👉 Go back to your browser tab and refresh the page!
